# Chapter 26 - Nonlinear Regression Models

### Steve Elston

Until now, we have only worked with Normally distributed **numeric response variables**. However, **nonlinear response variables** are extremely common. For example, **categorical responses** and responses of counts from **point processes**.     

In this notebook you will be introduced to working with models with nonlinear response variable. Specifically, you will work with the **generalized linear model (GLM)**. The GLM transforms a nonlinear response to a linear, or Normally distributed response using a **link function**. Link functions exist for any response distribution which can be written in exponential form.        

We solve the nonlinear problem by creating a solution to a linear model. The nonlinear response is then computed using the **inverse link function**.       

## Preparing the Data Set

In this section we will prepare the data set used in this notebook. These data were compiled from credit data for German bank customers, and are from the [University of California Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data). The objective is to build a model to predict customer credit quality, [good,bad], using the variable provided. This is a notoriously difficult problem.    

> **Exercise 26-1:** The data sample is based on the outcome of customers fulfilling their repayment obligations on loans. However, as is typically the case, we have no information on how perspective customers who were turned down for loads and therefore have no payment history. The outcomes for these people are **unobservable**. Write a few sentences discussing how any model fitted with these data could lead to biased credit decisions.    

As a first step, execute the code in the cell below to import the required packages.

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as nr
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as ss
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import sklearn.metrics as sklm
from patsy import dmatrices

%matplotlib inline

Execute the cell below to display the dimensions of the data frame and display the data types of the columns. 

In [ ]:
credit_data = pd.read_csv('../data/German_Credit_Clean.csv')
print(credit_data.shape)
credit_data.dtypes

There are a number of numeric and categorical predictor variables. The label is binary with values $[0,1]$, of type 'int64'. We will discuss the properties of this label further shortly.  

One difficulty with this data set is that there is an **imbalance** between the negative cases and the positive cases.

In [ ]:
np.sum(credit_data.loc[:,'bad_credit'])

The number of positive cases is only 30% of all cases. This imbalance will bias the training of any model toward the negative cases.

Now, we will prepare training and test data sets with normalized numeric features (predictor variables). Execute this code to create Bernoulli sampled training and test data sets. 

In [ ]:
num_cols = ['loan_duration_mo','loan_amount','payment_pcnt_income','age_yrs']
credit_data.loc[:,num_cols] = normalize(credit_data.loc[:,num_cols])

## Create a mask and use it to split the data into a train and test set   
nr.seed(6665)
mask = nr.choice(credit_data.index, size = 600, replace=False)
credit_data_train = credit_data.iloc[mask,:]
credit_data_test = credit_data.drop(mask, axis=0) 

## Logistic Regression

Until now, we have been working strictly with linear regression models. Linear regression models have a numeric label. Further, ordinary linear regression assumes the label values are Normally distributed. 

But, what if the label has another distribution, particularly if the label is categorical? Using a more general form for the label values leads us to the **generalized linear model**. Generalized linear model can use a number of distributions for the label .   

Here, we will look at a widely used variation of the generalized linear model using a **Binomial distribution**. This method is commonly known as **logistic regression**.

Logistic regression is widely used as a classification model. Logistic regression is a linear model, with a binary response or label values, `{False, True}` or `{0, 1}`.  specifically, the response is computed as a log likelihood, leading to a Binomial distribution of the label values. 

The response of the linear model is transformed to the log likelihood using a sigmoidal function, also know as the **logistic function** or **logit**:

$$f(x) = \frac{1}{1 + e^{-\kappa(x - x_0)}} \\
\kappa = steepness$$

Execute the code in the cell below to compute and plot an example of the logistic function.

In [ ]:
# Plot the logistic transformation function (f(x) above)
x_seq = np.linspace(-7, 7, 100)

def log_fun(x, center=0, scale=1):
    e = np.exp(scale*(center-x))
    log_out = 1./(1. + e)
    return log_out

log_fun_vectorized = np.vectorize(log_fun)

log_y = log_fun_vectorized(x_seq)

plt.plot(x_seq, log_y)
plt.title('Standard Logistic Function')
plt.xlabel('x')
plt.ylabel('Probability')
plt.grid()

Let's make this a bit more concrete with a simple example. Say we have a linear model:

$$\hat{y} = \beta_0 + \beta_1\ x$$

Now, depending on the value of $\hat{y}$ we want to classify the output from a logistic regression model as either `0` or `1`. We can use the linear model in the logistic function as follows:

$$F(\hat{y}) = \frac{1}{1 + e^{-\kappa(\beta_0 + \beta_1\ x)}} $$

In this way we transform the continious output of the linear model defined on $-\infty \le \hat{y} \le \infty$ to a binary response, $0 \le F(\hat{y}) \le 1$

The aforementioned function maps the output of a linear model to the response values of the label. This function is known as the **link function**, since it links the linear response to the label values.  

### What is Deviance?

The significance of the GLM is expressed in terms of something called **deviance**. It can be a bit of a challenge to wrap your head around what deviance really means. It turns out that you already know one form of deviance. The familiar mean squared error of a model with Normally distributed residuals is the deviance of this model.  

For the model above the residuals would be Binomially distributed. We need a more general formulation of deviance for such a model. For a model with label vector $y$, predictions$\hat{\mu}$, and estimated model parameters, $\hat{\theta}$ the general form of deviance can be written:

\begin{align}
D(y,\hat{\mu}) &= 2 \Big( log \big( p(y | \hat{\theta}_{S}) \big) - log \big( p(y | \hat{\theta}_{0}) \big) \Big) \\
&= 2 \Big( \mathcal{l}(y | \hat{\theta}_{S})  -  \mathcal{l}(y | \hat{\theta}_{0})  \Big)
\end{align}

where,    
- $\mathcal{l}(y | \hat{\theta}) \big)$ is the log likelihood of a model with estimated parameters, $\hat{\theta}$, given labels $y$.    
- $\hat{\theta}_{S}$ are the estimated parameters of a **saturated**; a hypothetical model, $M_S$, with a parameter for each observation, and therefore having the best possible fit to the training data.   
- $\hat{\theta}_{0}$ are the actual estimated parameters of the model, $M_0$, that we wish to evaluate.   

This all seems rather abstract. In particular, what use is this hypothetical saturated model? 

Fortunately, we can use the **deviance ratio** to compare models in a form we can actually work with. The trick is to recognize that the log likelihood of the saturated model, $\mathcal{l}(y | \hat{\theta}_{0})$, is just a constant. With this insight we can rewrite the deviance as:

$$D(y,\hat{\mu}) = Constant + 2 \mathcal{l}(y | \hat{\theta}_{0})$$

This looks a bit better! Now we can work with the deviance ratio to compare two models, $M_1$ and $M_0$:   

\begin{align}
D(\hat{\theta}_1,\hat{\theta}_0) &= \frac{Constant + 2 \mathcal{l}(y | \hat{\theta}_{0})}{Constant + 2 \mathcal{l}(y | \hat{\theta}_{1})} \\ 
&= Constant + 2 \mathcal{l}(y | \hat{\theta}_{0}) - Constant - 2 \mathcal{l}(y | \hat{\theta}_{1}) \\
&= 2 \mathcal{l}(y | \hat{\theta}_{0}) - 2 \mathcal{l}(y | \hat{\theta}_{1})
\end{align}

Which model should we use as a comparison in the above relation? The **null model** is a good choice. The null model is just a model that makes predictions based on the expected value of mean. This formulation is called the **null deviance**, and is what is typically displayed by most statistical software packages, including statsmodels. Unfortunately, this quantity is usually just called, deviance, even when it is actually null deviance, something rather different.   

A nice property of null deviance is that for large number of samples (approaching infinity), it is Chi Squared distributed. Thus, we can test the statistical significance of the model against the null model using the Chi Squared test. Since the derivation of this test depends on something called Wilk's theorem, this is often referred to as **Wilk's test**. 

The null deviance is also useful for comparing models. If you have two models and want to know which is better, pick the one with the maximum null deviance.  

## Classification Example

To make these concepts concrete we will now build and test a classification model with Binomial response. To understand the features and labels of this model it will be useful to look at the model matrix and the label column. Execute the code in the cell below to display the head of the label array and the model matrix. 

In [ ]:
formula = 'bad_credit ~ C(credit_history) + C(purpose) + C(gender_status) + C(time_employed_yrs) +C(other_credit_outstanding)\
           + C(property) + C(job_category) + loan_duration_mo + payment_pcnt_income + age_yrs'
Y, X = dmatrices(formula, data=credit_data_train)
print(Y[:5])
print(X[:2])

The label array is binary with values $[0,1$. This response is must be Binomially distributed making this a nonlinear response problem.  

The feature (design) matrix is not surprising. The first column contains the intercept term of all 1s. The categorical feature columns are all one-hot encoded. The last three columns are the normalized values of the numeric features. 

The statsmodels [GLM](https://www.statsmodels.org/stable/examples/notebooks/generated/glm.html) function implements the [generalize linear model](https://www.statsmodels.org/stable/glm.html#families) for a number of response distributions. The code in the cell below uses the Binomial family as the distribution of the label. Execute this code and examine the model summary. 

In [ ]:
nr.seed(4536)
glm_model = smf.glm(formula, data=credit_data_train, family=sm.families.Binomial()).fit()
glm_model.summary()

Examine the summary of the model noticing the following:    
- The distribution family of the label is Binomial as expected. 
- The **link function** is the logit. The link function transforms the results of the linear model to the label distribution. 
- The Pearson Chi Squared statistic shows that the model is significant compared to a null model based on the difference in deviance.   
- The standard errors, confidence intervals and p-values show that most of the features are not significant. In other words, we cannot reject the null hypothesis that many of these coefficients have a value of 0.  

### From Probability to Class

The predictions from a generalized linear model are the inverse link function transformed values of the linear model. This output or response in in the from of the probability distribution chosen. For our example, the response is Binomially distributed. The probability of the response (1) is computed using the **logit function** as the transformation.     

In order to generate binary predictions (good or bad credit), we need to decide on a cutoff value for the probability. The cutoff value separates the two cases. Probabilities below the cutoff are classified as the negative outcome (good credit). Probabilities above the cutoff are classified as the positive outcome (bad credit). In effect, applying a cutoff in this manner is a **one-sided hypothesis test**; can you reject the null case and find a positive case?     

Commonly, we choose 0.5 as a cutoff.  But  this choice is arbitrary and we can set it to whatever probability we choose, given the problem at hand. If we have a model where False Positives are costly, we might decide to increase the cutoff, and vice-versa if False Negatives are more costly. As an example, cost to the bank from the loss from issuing a loan to a bad credit customer might well exceed the cost of not issuing a loan to a good credit customer. The later cost might include the cost of loss of business and the customer service costs of correcting the status of otherwise good customers. 

The code in the cell below does the following:    
1. Computes the probabilities of the response variables using the `predict` method.      
2. The probabilities are transformed from probabilities to the classes using the cutoff value.   
3. A column with the classes of the label are created from the binary values.   

Execute the code and examine the results.  

In [ ]:
import warnings  

threshold = 0.5
def score_model(df, model, threshold):
    warnings.filterwarnings("ignore")
    df.loc[:,'prediction_probs'] = model.predict(df)
    df.loc[:,'predicted_credit'] = ['bad' if x > threshold else 'good' for x in df.loc[:,'prediction_probs']]
    df.loc[:,'credit_actual'] = ['bad' if x==1 else 'good' for x in df.loc[:,'bad_credit']]
    warnings.resetwarnings()
    return df

credit_data_train = score_model(credit_data_train, glm_model, threshold)
credit_data_train.head(20)

Compare the `predicted_credit` and `credit_actual` columns. Notice that the prediction agrees with the actual value in several cases. However, there are incorrect predictions. Thinking in terms of a hypothesis test:   
1. **Type I errors** occur when a true negative case is classified as positive. The null hypothesis is incorrectly rejected.   
2. **Type II errors** occur when a true positive case is classified as negative. There is a failure to reject the null hypothesis.   

> **Exercise 26-2:** For the training data you can create a plot showing the cumulative probabilities computed for each of the classes as follows:   
> 1. Use the `seaborn.ecdfplot` function (empirical CDF) with `hue='bad_credit'` to display the CDF of the `prediction_probs`. 
> 2. Use the matplotlib axis returned to add a vertical red dotted line to the plot at the threshold value.  

In [ ]:
## Your code goes here


> Examine the plot you have created. The fractions of correct and incorrect classifications (decisions) can be read on the plot from the proportion above or below the crossing point between each CDF and the threshold. For example, the number of Type II errors is proportion of the positive curve to the left of the threshold. Given the threshold value, do you expect a large number of Type II errors and a smaller number of Type I errors, and why?   
> **End of exercise.**

### Evaluating the model   

How can we evaluate the performance of a classifier model? As with any statistical or machine learning model there is no one metric which can be considered 'best'. We will consider only a few of the possibilities here.  

First, let us consider all the possible outcomes of the hypothesis test. 

 - **True Positives (TP):** We predicted bad credit and the credit is bad.
 - **True Negatives (TN):** We predicted good credit and the credit is good.
 - **False Positives (FP):** We predicted bad credit but the credit was good; a **Type I error**.
 - **False Negatives (FN):** We predicted good credit but the credit was bad; a **Type II error**.
 
For the Binomial case, we can lay these options out in a **truth table**, which is commonly called a **confusion matrix**:
 
 |  | Classified Positive | Classified Negative |
| ---- | :---: | :---: |
|Positive | TP | FN |  
|Negative | FP | TN |

Some commonly used metrics are computed directly from the foregoing quantities:

- **Accuracy** = (TP + TN) / (TP + FP + TN + FN)
- **Precision** = TP / (TP + FP)
   - Precision is the correct fraction of the positive relevant class predictions.
- **Recall** = TP / (TP + FN)
   - Recall is the correct fraction of the relevant positive class members predicted.
- **F1** = $\frac{2\ precision\ recall}{precision + recall}$ = $\frac{TP}{TP + 1/2(FP +FN)}$      
   - Is the **harmonic mean** between precision and recall. In other words, a statistic which trades-offs precision and recall. 
   
These metrics can be computed using the [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics) package. In this case we use some of the classification metrics. Now, execute the code in the cell below to display performance metrics for the classifier model using the training data.    

In [ ]:
def print_metrics(df):
    labels = df.loc[:,'credit_actual']
    scores = df.loc[:,'predicted_credit']
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])
    
print_metrics(credit_data_train) 

Examine these metrics which can be interpreted as follows:   
1. A perfect classifier would produce a confusion matrix with entries only on the diagonal; no errors. In this case, there are significant errors. The number of false positives compared to the total number of positives is relatively low. However, fewer that half of the positive cases have been correctly classified.    
2. The overall accuracy of the model is reasonable.  
3. However, the recall statistic shows a significant problem with the model. The positive case recall is only 0.40, consistent with the low fraction of the correctly classified positive cases.   
4. Likewise, the precision is lower for positive cases as well.     

In summary, the initial classifier model is doing a poor job of detecting bad credit cases. In part, this performance results from the imbalance between the number of positive and negative cases in the data set.    

The evaluation of the model based on training data is undoubtedly biased, since this data was used to fit the model. Using the **held-back test data** is a more objective approach to model evaluation. Execute the code in the cell below to evaluated the model using the test data.       

In [ ]:
credit_data_test = score_model(credit_data_test, glm_model, threshold)
print_metrics(credit_data_test) 

Overall, these results are consistent with those obtained using the training data. However, the results obtained with the test data are a bit worse than for the training data. This is expected, since the test data are a random sample of the original data.   

### Finding a better threshold

The forgoing results are not good. Since the choice of threshold was arbitrary, it is likely that another choice could yield better results. A hyperparameter search could find a threshold producing a better result.    

The question is, which metric should be used to optimize model performance? Accuracy, can be misleading, particularly in cases with imbalanced label values. For this problem we will assume that the bank will be more concerned with identifying potential bad credit customers. These considerations make recall seem to be a reasonable metric. In this case we will use F1, since it trades-off precision and recall.        

> **Exercise 26-3:** The code in the cell below performs a hyperparameter search of the cutoff threshold measured using the F1 score for the positive cases. Execute this code and examine the chart displayed.      

In [ ]:
def threshold_test(model, df_train, df_test, thresholds, formula=formula):
    '''Function that tests accuracy of classifier for different threshold values'''
    accuracy_train = []
    accuracy_test = []
    for i,threshold in enumerate(thresholds):
        ## Compute the training accuracy
        df_train = score_model(df_train, model, threshold)
        accuracy_train.append(sklm.precision_recall_fscore_support(df_test.loc[:,'credit_actual'], df_test.loc[:,'predicted_credit'])[2][0])
        ## Then compute the test accuracy
        df_test = score_model(df_test, model, threshold)
        accuracy_test.append(sklm.precision_recall_fscore_support(df_test.loc[:,'credit_actual'], df_test.loc[:,'predicted_credit'])[2][0])      
    return accuracy_train, accuracy_test


def plot_thresholds(thresholds, accuracy_train, accuracy_test):
    fig, ax = plt.subplots(figsize=(8, 5)) # define axis  
    ax.plot(thresholds, accuracy_train, label='Training F1')
    ax.plot(thresholds, accuracy_test, label='Test F1')
    ax.set_ylabel('F1')
    ax.set_xlabel('Threshold value')
    ax.set_title('F1 vs. decision threshold')
    plt.legend()
    plt.show()
    
thresholds = np.arange(0.2, 0.8, step = 0.05)
accuracy_train, accuracy_test = threshold_test(glm_model, credit_data_train, credit_data_test, thresholds)
plot_thresholds(thresholds, accuracy_train, accuracy_test)    

> Examine your plot. What does this downward slope of the F1 curve with threshold value tell you about the trade-off between F1 and threshold? How is this trade-off dependent on the cost of each type of error for the problem at hand?    
> **End of exercise.**    

> **Exercise 26-4:** You will now test how the model evaluation changes when using a threshold value to 0.35. While this choice is a bit arbitrary, the goal is to improve the performance at detecting positive cases by reducing Type II errors at the expense of Type I errors with negative cases. Do the following:   
> 1. In the first cell below set the threshold value, then score the model and print the metrics using the training data.  
> 2. In the second cell below use the new threshold value to score the model and print the metrics using the test data.  

In [ ]:
## Your code goes here


In [ ]:
## Your code goes here


> Compare the performance metrics for the training and testing data with the original threshold value of 0.5 with those of the updated threshold of 0.35.   
> 1. How have the errors shown in the confusion matrices changed?   
> 2. How have the recall and precision changed between the positive and negative cases and what does this tell you about the trade-off between Type I and Type II errors?  
> **End of exercise.**

For the testing data the plot below shows the probabilities computed for each of the classes using the testing data. This plot is the same type you created in Exercise 26-2.   

In [ ]:
ax = sns.ecdfplot(x='prediction_probs', hue='bad_credit', data=credit_data_train) 
ax.axvline(0.35, color='red', linestyle='dotted')

Compare this plot to the one created in Exercise 26-2. Notice how the number of Type II errors arising from the positive case curve will be lower. This improvement is at the expense of Type I errors arising from the negative curve.  

### Applying regularization      

So far, we have been working with an over-fit model. Now we will explore the application of regularization to the model. The code in the cell below computes the training and test F1 statistics of the model as a function of the elastic net regularization parameter $\alpha$. The training and test F1 statistics are then plotted as a function of $\alpha$. Execute this code to execute the computation and display the plot.      

In [ ]:
def regularized_coefs(df_train, df_test, alphas, n_coefs=31,
                      formula = formula, threshold=threshold):
    '''Function that computes a linear model for each value of the regualarization 
    parameter alpha and returns an array of the coefficient values. The L1_wt 
    determines the trade-off between L1 and L2 regualarization'''
    coefs = np.zeros((len(alphas),n_coefs + 1))
    accuracy_train = []
    accuracy_test = []
    for i,alpha in enumerate(alphas):
        ## First compute the training MSE
        #### Complete the line of code below
        temp_mod = smf.glm(formula, data=df_train).fit_regularized(method='elastic_net', alpha=alpha)
        ## Compute the training accuracy
        df_train = score_model(df_train, temp_mod, threshold)
        coefs[i,:] = temp_mod.params
        # accuracy_train.append(sklm.accuracy_score(df_test.loc[:,'credit_actual'], df_test.loc[:,'predicted_credit']))
        accuracy_train.append(sklm.precision_recall_fscore_support(df_test.loc[:,'credit_actual'], df_test.loc[:,'predicted_credit'])[2][0])
        ## Then compute the test accuracy
        df_test = score_model(df_test, temp_mod, threshold)
#        accuracy_test.append(sklm.accuracy_score(df_test.loc[:,'credit_actual'], df_test.loc[:,'predicted_credit'])) 
        accuracy_test.append(sklm.precision_recall_fscore_support(df_test.loc[:,'credit_actual'], df_test.loc[:,'predicted_credit'])[2][0])      
    return coefs, accuracy_train, accuracy_test


def plot_coefs(coefs, alphas, accuracy_train, accuracy_test, ylim=None):
    fig, ax = plt.subplots(1,2, figsize=(12, 5)) # define axis
    for i in range(coefs.shape[1]): # Iterate over coefficients
        ax[0].plot(alphas, coefs[:,i])
    ax[0].axhline(0.0, color='red', linestyle='--', linewidth=0.5)
    ax[0].set_ylabel('Partial slope values')
    ax[0].set_xlabel('alpha')
    ax[0].set_title('Parial slope values vs. regularization parameter number')
    if ylim is not None: ax[0].set_ylim(ylim)
    
    ax[1].plot(alphas, accuracy_train, label='Training F1')
    ax[1].plot(alphas, accuracy_test, label='Test F1')
    ax[1].set_ylabel('F1')
    ax[1].set_xlabel('alpha')
    ax[1].set_title('F1 vs. regularization parameter')
    plt.legend()
    plt.show()   
    
threshold = 0.35
alphas = np.arange(0.0, 0.002, step = 0.00005)
#alphas = np.arange(0.0, 0.05, step = 0.0005)
Betas, accuracy_train, accuracy_test = regularized_coefs(credit_data_train, credit_data_test, alphas)
plot_coefs(Betas, alphas, accuracy_train, accuracy_test, ylim=[-0.3,0.5])    

The results are a bit erratic looking, but consistent with what is expected. The model coefficients can been seen being constrained and in some cases driven toward 0 by the elastic net regularization as the hyperparameter increases. The curve of the training and test errors is rather jagged. However, one can see that a reasonable choice of hyperparameter is around 0.0007.    

> **Exercise 26-5:** Using the regularization parameter identified and the threshold value of 0.35 you will now evaluate the model. In the cells below you will create and execute code to evaluate the model using first the training data and then the testing data by these steps:   
> 1. Fit the glm model using elastic net regularization and $\alpha = 0.0007$. You should fit the model once with the training data.   
> 2. Score the model using $threshold = 0.35$ using either the training of testing data.   
> 3. Compute and print the model evaluation metrics.  

In [ ]:
## Your code goes here


In [ ]:
## Your code goes here


> The goal of regularization is not to improve model training performance, but rather to improve generalization performance. Keeping the foregoing in mind, compare the performance metrics for the training and testing data of the OLS model with threshold value of 0.35 with those of the regularized model with the same threshold value.   
> 1. Have the errors shown in the confusion matrices changed for either the training of testing data in a way that would indicate model performance is significantly reduced?   
> 2. How have the recall and precision changed between the positive and negative cases and what does this change tell you about the Type I and Type II generalization errors when regularization is applied?
> **End of exercise.**

> **Exercise 26-6:** as a final step in comparing the coefficients of the OLS and regularized models. Execute the code in the cell below to display the coefficient values for the regularized model.

In [ ]:
glm_model_regularized.params

> Compare these coefficient values to the ones displayed by the summary method for the OLS model.    
> 1. How well are the few very large coefficient values computed for the OLS model constrained by the regularization? Pay particular attention to the coefficients of the numeric variables and at least one categorical dummy variable.  
> 2. Are some of the coefficients for both numeric and categorical dummy variables pushed to exactly zero, and what does this tell you about model selection? 

#### Copyright 2017, 2018, 2019, 2020, 2021 Stephen F Elston. All rights reserved. 